In [1]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime

# Inicializar MediaPipe Pose y Hands
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Configurar detectores
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity=1  # 0=ligero, 1=medio, 2=pesado
)

hands = mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands=2
)

# Captura de video
cap = cv2.VideoCapture(0)  # 0 = webcam por defecto

print("Presiona 'q' para salir")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertir BGR a RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Detectar pose y manos
    pose_results = pose.process(image)
    hands_results = hands.process(image)
    
    # Volver a BGR para dibujar
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Dibujar landmarks de pose
    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
        )
    
    # Dibujar landmarks de manos
    if hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
    
    # Mostrar FPS
    cv2.putText(image, f'FPS: {int(cap.get(cv2.CAP_PROP_FPS))}', 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('MediaPipe Pose + Hands', image)
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pose.close()
hands.close()


Presiona 'q' para salir


In [2]:
def extract_pose_data(pose_landmarks):
    """Extrae coordenadas de landmarks de pose"""
    if not pose_landmarks:
        return None
    
    landmarks = []
    for landmark in pose_landmarks.landmark:
        landmarks.append({
            'x': landmark.x,
            'y': landmark.y,
            'z': landmark.z,
            'visibility': landmark.visibility
        })
    return landmarks

def extract_hand_data(hands_results):
    """Extrae coordenadas y clasificación de manos"""
    if not hands_results.multi_hand_landmarks:
        return None
    
    hands_data = []
    for idx, hand_landmarks in enumerate(hands_results.multi_hand_landmarks):
        hand_info = {
            'handedness': hands_results.multi_handedness[idx].classification[0].label,  # "Left" o "Right"
            'score': hands_results.multi_handedness[idx].classification[0].score,
            'landmarks': []
        }
        
        for landmark in hand_landmarks.landmark:
            hand_info['landmarks'].append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z
            })
        
        hands_data.append(hand_info)
    
    return hands_data

# Testing extracción
print("Ejemplo de landmarks de pose:", extract_pose_data(pose_results.pose_landmarks)[:3])
print("Ejemplo de landmarks de manos:", extract_hand_data(hands_results))


Ejemplo de landmarks de pose: [{'x': 0.6356474757194519, 'y': 0.5357095003128052, 'z': -1.2878602743148804, 'visibility': 0.9997855424880981}, {'x': 0.653036892414093, 'y': 0.45773619413375854, 'z': -1.1983544826507568, 'visibility': 0.9996163845062256}, {'x': 0.6700222492218018, 'y': 0.45641347765922546, 'z': -1.1983556747436523, 'visibility': 0.9996852874755859}]
Ejemplo de landmarks de manos: None


In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
from dataclasses import dataclass
import time

# ==================== INICIALIZACIÓN MEDIAPIPE ====================
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Configuración de detectores
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.5
)

face_detection = mp_face.FaceDetection(
    min_detection_confidence=0.6
)

# ==================== SISTEMA DE PLANOS ====================
PLANOS = {
    'PLANO_GENERAL': {'zoom': 1.0, 'nombre': 'Plano General'},
    'PLANO_MEDIO': {'zoom': 1.5, 'nombre': 'Plano Medio'},
    'PRIMER_PLANO': {'zoom': 2.0, 'nombre': 'Primer Plano'},
    'PLANO_DETALLE': {'zoom': 2.5, 'nombre': 'Plano Detalle'}
}

# ==================== DETECCIÓN DE GESTOS ====================
def contar_dedos(hand_landmarks):
    """Cuenta dedos levantados"""
    dedos = 0
    tips_ids = [4, 8, 12, 16, 20]
    
    # Pulgar (comparación horizontal según lado de la mano)
    if hand_landmarks.landmark[tips_ids[0]].x < hand_landmarks.landmark[tips_ids[0] - 1].x:
        dedos += 1
    
    # Otros dedos (comparación vertical)
    for i in range(1, 5):
        if hand_landmarks.landmark[tips_ids[i]].y < hand_landmarks.landmark[tips_ids[i] - 2].y:
            dedos += 1
    
    return dedos

def detectar_gesto_puño(hand_landmarks):
    """Detecta puño cerrado"""
    return contar_dedos(hand_landmarks) == 0

def detectar_gesto_paz(hand_landmarks):
    """Detecta gesto de paz (V)"""
    indice_up = hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y
    medio_up = hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y
    anular_down = hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y
    meñique_down = hand_landmarks.landmark[20].y > hand_landmarks.landmark[18].y
    
    return indice_up and medio_up and anular_down and meñique_down

def clasificar_gesto(hand_landmarks):
    """Clasifica gesto y retorna plano correspondiente"""
    dedos = contar_dedos(hand_landmarks)
    
    if detectar_gesto_puño(hand_landmarks):
        return 'PLANO_GENERAL', dedos
    elif dedos == 1:
        return 'PLANO_MEDIO', dedos
    elif detectar_gesto_paz(hand_landmarks) or dedos == 2:
        return 'PRIMER_PLANO', dedos
    elif dedos >= 3:
        return 'PLANO_DETALLE', dedos
    else:
        return 'PLANO_GENERAL', dedos

# ==================== ORIENTACIÓN CORPORAL ====================
class BodyOrientation:
    """Calcula orientación corporal 3D desde pose landmarks"""
    def __init__(self):
        self.history = deque(maxlen=8)
    
    def calculate(self, pose_landmarks):
        """Calcula yaw, pitch, roll del cuerpo"""
        if not pose_landmarks:
            return None
        
        lm = pose_landmarks.landmark
        
        # Puntos clave para orientación
        left_shoulder = np.array([lm[11].x, lm[11].y, lm[11].z])
        right_shoulder = np.array([lm[12].x, lm[12].y, lm[12].z])
        left_hip = np.array([lm[23].x, lm[23].y, lm[23].z])
        right_hip = np.array([lm[24].x, lm[24].y, lm[24].z])
        
        # Vector de hombros (yaw - rotación horizontal)
        shoulder_vec = right_shoulder - left_shoulder
        yaw = np.arctan2(shoulder_vec[2], shoulder_vec[0]) * 180 / np.pi
        
        # Inclinación lateral (roll)
        roll = np.arctan2(shoulder_vec[1], shoulder_vec[0]) * 180 / np.pi
        
        # Inclinación adelante/atrás (pitch)
        torso_center = (left_shoulder + right_shoulder) / 2
        hip_center = (left_hip + right_hip) / 2
        torso_vec = torso_center - hip_center
        pitch = np.arctan2(torso_vec[2], torso_vec[1]) * 180 / np.pi
        
        orientation = {
            'yaw': yaw,
            'pitch': pitch,
            'roll': roll,
            'visibility': min(lm[11].visibility, lm[12].visibility)
        }
        
        self.history.append(orientation)
        return self._smooth()
    
    def _smooth(self):
        """Suavizado temporal"""
        if not self.history:
            return None
        return {
            'yaw': np.mean([o['yaw'] for o in self.history]),
            'pitch': np.mean([o['pitch'] for o in self.history]),
            'roll': np.mean([o['roll'] for o in self.history]),
            'visibility': np.mean([o['visibility'] for o in self.history])
        }

# ==================== SEGUIMIENTO Y ENCUADRE ====================
@dataclass
class FrameTarget:
    """Target de encuadre con transición suave"""
    x: float = 0.5
    y: float = 0.5
    zoom: float = 1.0

class SmoothFramer:
    """Sistema de encuadre con transiciones suaves"""
    def __init__(self, smoothing=0.12):
        self.current = FrameTarget()
        self.target = FrameTarget()
        self.smoothing = smoothing
    
    def update(self, center_x, center_y, zoom):
        """Actualiza target y suaviza transición"""
        self.target.x = center_x
        self.target.y = center_y
        self.target.zoom = zoom
        
        # Interpolación suave
        self.current.x += (self.target.x - self.current.x) * self.smoothing
        self.current.y += (self.target.y - self.current.y) * self.smoothing
        self.current.zoom += (self.target.zoom - self.current.zoom) * self.smoothing
        
        return self.current

def obtener_centro_seguimiento(pose_landmarks, face_result, w, h):
    """
    Obtiene punto de seguimiento inteligente:
    1. Cara si está disponible
    2. Punto medio entre nariz y hombros
    3. Centro del frame
    """
    # Prioridad 1: Cara detectada
    if face_result and face_result.detections:
        bbox = face_result.detections[0].location_data.relative_bounding_box
        cx = bbox.xmin + bbox.width / 2
        cy = bbox.ymin + bbox.height / 2
        return (cx, cy)
    
    # Prioridad 2: Pose landmarks (cabeza/torso superior)
    if pose_landmarks:
        lm = pose_landmarks.landmark
        # Promedio de nariz y punto medio de hombros
        nose = np.array([lm[0].x, lm[0].y])
        shoulders = (np.array([lm[11].x, lm[11].y]) + np.array([lm[12].x, lm[12].y])) / 2
        center = (nose + shoulders) / 2
        return (float(center[0]), float(center[1]))
    
    # Default: centro del frame
    return (0.5, 0.5)

def aplicar_encuadre(frame, framer_state):
    """
    Aplica zoom y reencuadre DESPUÉS de dibujar landmarks
    Esto soluciona el problema de landmarks desalineados
    """
    h, w = frame.shape[:2]
    
    # Calcular región de interés
    zoom = framer_state.zoom
    crop_w = int(w / zoom)
    crop_h = int(h / zoom)
    
    # Centro normalizado -> píxeles
    center_x = int(framer_state.x * w)
    center_y = int(framer_state.y * h)
    
    # Calcular límites del crop
    x1 = max(0, center_x - crop_w // 2)
    y1 = max(0, center_y - crop_h // 2)
    x2 = min(w, x1 + crop_w)
    y2 = min(h, y1 + crop_h)
    
    # Ajustar si sale de límites
    if x2 - x1 < crop_w:
        x1 = max(0, x2 - crop_w)
    if y2 - y1 < crop_h:
        y1 = max(0, y2 - crop_h)
    
    # Crop y resize
    cropped = frame[y1:y2, x1:x2]
    if cropped.size == 0:  # Validación
        return frame
    
    resized = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
    return resized

# ==================== VISUALIZACIÓN ====================
def dibujar_landmarks(frame, pose_results, hands_results, face_results):
    """
    Dibuja todos los landmarks EN EL FRAME ORIGINAL
    Esto es CRÍTICO para evitar desalineación
    """
    # Dibujar pose
    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
        )
    
    # Dibujar manos
    if hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
    
    # Dibujar cara
    if face_results.detections:
        for detection in face_results.detections:
            mp_drawing.draw_detection(frame, detection)
    
    return frame

def dibujar_info_deteccion(frame, num_dedos, plano, orientation):
    """Dibuja info en frame de detección (izquierdo)"""
    h, w = frame.shape[:2]
    y_offset = 30
    
    # Dedos detectados
    if num_dedos is not None:
        cv2.putText(frame, f"Dedos: {num_dedos}", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        y_offset += 35
    
    # Plano actual
    cv2.putText(frame, f"Plano: {PLANOS[plano]['nombre']}", (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    y_offset += 35
    
    # Orientación corporal
    if orientation and orientation['visibility'] > 0.5:
        cv2.putText(frame, f"Yaw: {orientation['yaw']:.1f}°", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 100), 2)
        y_offset += 28
        cv2.putText(frame, f"Pitch: {orientation['pitch']:.1f}°", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 100), 2)
        y_offset += 28
        cv2.putText(frame, f"Roll: {orientation['roll']:.1f}°", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 100), 2)
    
    return frame

def dibujar_info_principal(frame, plano, fps):
    """Dibuja info en frame principal (derecho)"""
    h, w = frame.shape[:2]
    
    # Nombre del plano (abajo a la izquierda)
    cv2.putText(frame, PLANOS[plano]['nombre'], (10, h - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
    
    # FPS (arriba a la derecha)
    cv2.putText(frame, f"FPS: {fps:.1f}", (w - 150, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Regla de tercios (grid)
    color_grid = (80, 80, 80)
    cv2.line(frame, (w//3, 0), (w//3, h), color_grid, 1)
    cv2.line(frame, (2*w//3, 0), (2*w//3, h), color_grid, 1)
    cv2.line(frame, (0, h//3), (w, h//3), color_grid, 1)
    cv2.line(frame, (0, 2*h//3), (w, 2*h//3), color_grid, 1)
    
    return frame

def crear_pantalla_dividida(frame_principal, frame_deteccion):
    """Crea pantalla dividida horizontal"""
    h, w = frame_principal.shape[:2]
    
    # Redimensionar ambos frames
    frame_det_small = cv2.resize(frame_deteccion, (w // 2, h))
    frame_prin_small = cv2.resize(frame_principal, (w // 2, h))
    
    # Concatenar horizontalmente: detección | principal
    split_screen = np.hstack((frame_det_small, frame_prin_small))
    
    return split_screen

# ==================== MAIN ====================
def main():
    cap = cv2.VideoCapture(0)
    
    # Configurar resolución
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    # Sistemas
    orientation_tracker = BodyOrientation()
    framer = SmoothFramer(smoothing=0.12)
    fps_history = deque(maxlen=30)
    
    # Estado
    plano_actual = 'PLANO_GENERAL'
    num_dedos = None
    
    print("🎬 Sistema de Cámara Inteligente")
    print("Controles:")
    print("  ESC - Salir")
    print("  'r' - Reset encuadre")
    print("\nGestos:")
    print("  Puño (0 dedos) -> Plano General")
    print("  1 dedo -> Plano Medio")
    print("  2 dedos (paz) -> Primer Plano")
    print("  3+ dedos -> Plano Detalle")
    
    while cap.isOpened():
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        
        # Voltear para efecto espejo
        frame = cv2.flip(frame, 1)
        h, w = frame.shape[:2]
        
        # Convertir a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # ===== PROCESAR DETECCIONES =====
        pose_results = pose.process(rgb_frame)
        hands_results = hands.process(rgb_frame)
        face_results = face_detection.process(rgb_frame)
        
        # ===== FRAME DE DETECCIÓN (con landmarks) =====
        frame_deteccion = frame.copy()
        frame_deteccion = dibujar_landmarks(
            frame_deteccion, pose_results, hands_results, face_results
        )
        
        # ===== ANALIZAR GESTOS Y ORIENTACIÓN =====
        # Gestos de mano
        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                plano_detectado, num_dedos = clasificar_gesto(hand_landmarks)
                plano_actual = plano_detectado
        
        # Orientación corporal
        orientation = orientation_tracker.calculate(pose_results.pose_landmarks)
        
        # ===== CALCULAR ENCUADRE =====
        centro = obtener_centro_seguimiento(pose_results.pose_landmarks, face_results, w, h)
        zoom_factor = PLANOS[plano_actual]['zoom']
        framer_state = framer.update(centro[0], centro[1], zoom_factor)
        
        # ===== FRAME PRINCIPAL (aplicar encuadre) =====
        # IMPORTANTE: Dibujar landmarks ANTES de aplicar zoom/crop
        frame_principal = frame.copy()
        frame_principal = dibujar_landmarks(
            frame_principal, pose_results, hands_results, face_results
        )
        # Ahora sí aplicar encuadre
        frame_principal = aplicar_encuadre(frame_principal, framer_state)
        
        # ===== DIBUJAR INFO =====
        frame_deteccion = dibujar_info_deteccion(
            frame_deteccion, num_dedos, plano_actual, orientation
        )
        
        fps = 1.0 / (time.time() - start_time)
        fps_history.append(fps)
        avg_fps = np.mean(fps_history)
        
        frame_principal = dibujar_info_principal(frame_principal, plano_actual, avg_fps)
        
        # ===== PANTALLA DIVIDIDA =====
        pantalla_final = crear_pantalla_dividida(frame_principal, frame_deteccion)
        
        # ===== MOSTRAR =====
        cv2.imshow('Sistema de Cámara Inteligente | Detección + Resultado', pantalla_final)
        
        # ===== CONTROLES =====
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break
        elif key == ord('r'):
            framer = SmoothFramer(smoothing=0.12)
            print("🔄 Encuadre reseteado")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    pose.close()
    hands.close()
    face_detection.close()

if __name__ == "__main__":
    main()


🎬 Sistema de Cámara Inteligente
Controles:
  ESC - Salir
  'r' - Reset encuadre

Gestos:
  Puño (0 dedos) -> Plano General
  1 dedo -> Plano Medio
  2 dedos (paz) -> Primer Plano
  3+ dedos -> Plano Detalle
